In [7]:
from PIL import Image
from numpy import asarray
import numpy as np
import os
import keras
from keras.layers import Dense, Conv2D, BatchNormalization, Activation
from keras.layers import AveragePooling2D, Input, Flatten
from tensorflow.keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras import backend as K
from keras.models import Model
from keras.datasets import cifar10
import numpy as np
import os
from keras.utils import np_utils
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
# calculates f1 for 1:100 dataset with 95tp, 5fn, 55fp
from sklearn.metrics import f1_score
print('all imported')

all imported


In [8]:
def readImage(filePath):
    img = Image.open(filePath)
    
    size=(30,40)
    #resize image
    out = img.resize(size)

    # asarray() class is used to convert
    # PIL images into NumPy arrays
    numpydata = asarray(out)
    numpydata = np.repeat(numpydata[:, :, np.newaxis], 3, axis=2)
    # <class 'numpy.ndarray'>
    #print(type(numpydata))

    #  shape
    #print(numpydata.shape)
    return numpydata
print(readImage('data/disease/measles (1).png'))

[[[[ 76  84  95]
   [ 76  84  95]
   [ 76  84  95]]

  [[ 15  15  20]
   [ 15  15  20]
   [ 15  15  20]]

  [[  8   7  11]
   [  8   7  11]
   [  8   7  11]]

  ...

  [[ 16  14  25]
   [ 16  14  25]
   [ 16  14  25]]

  [[ 13  13  22]
   [ 13  13  22]
   [ 13  13  22]]

  [[  8   9  13]
   [  8   9  13]
   [  8   9  13]]]


 [[[ 88  96 107]
   [ 88  96 107]
   [ 88  96 107]]

  [[ 17  17  22]
   [ 17  17  22]
   [ 17  17  22]]

  [[  6   5   9]
   [  6   5   9]
   [  6   5   9]]

  ...

  [[ 15  13  23]
   [ 15  13  23]
   [ 15  13  23]]

  [[ 15  13  24]
   [ 15  13  24]
   [ 15  13  24]]

  [[ 10  10  16]
   [ 10  10  16]
   [ 10  10  16]]]


 [[[ 86  91  98]
   [ 86  91  98]
   [ 86  91  98]]

  [[ 14  13  17]
   [ 14  13  17]
   [ 14  13  17]]

  [[  5   4   7]
   [  5   4   7]
   [  5   4   7]]

  ...

  [[ 15  13  23]
   [ 15  13  23]
   [ 15  13  23]]

  [[ 15  13  23]
   [ 15  13  23]
   [ 15  13  23]]

  [[ 12  11  18]
   [ 12  11  18]
   [ 12  11  18]]]


 ...


 [[[157 162 

In [9]:
listFiles = os.listdir('data/disease/')
print(listFiles[:5])

['4164443_down-syndrome-down-syndrome-transparent-background-transparent-png.png', 'main-qimg-4c812ebe5c1bdc504625b6961032fd3b.png', 'measles (1).png', 'measles (10).jpg', 'measles (140).jpg']


In [10]:
listCancer = []
listResults = []
for elem in listFiles:
    img = readImage('data/disease/'+elem)
    listCancer += [img]
    listResults += [1]
print(listCancer[:3])
print(img.shape)

[array([[[[243, 243, 243],
         [243, 243, 243],
         [243, 243, 243]],

        [[242, 242, 242],
         [242, 242, 242],
         [242, 242, 242]],

        [[242, 242, 242],
         [242, 242, 242],
         [242, 242, 242]],

        ...,

        [[242, 242, 242],
         [242, 242, 242],
         [242, 242, 242]],

        [[242, 242, 242],
         [242, 242, 242],
         [242, 242, 242]],

        [[242, 242, 242],
         [242, 242, 242],
         [242, 242, 242]]],


       [[[242, 242, 242],
         [242, 242, 242],
         [242, 242, 242]],

        [[242, 242, 242],
         [242, 242, 242],
         [242, 242, 242]],

        [[242, 242, 242],
         [242, 242, 242],
         [242, 242, 242]],

        ...,

        [[242, 242, 242],
         [242, 242, 242],
         [242, 242, 242]],

        [[242, 242, 242],
         [242, 242, 242],
         [242, 242, 242]],

        [[242, 242, 242],
         [242, 242, 242],
         [242, 242, 242]]],


       

In [11]:
listFiles = os.listdir('data/health/')
print(listFiles[:5])
for elem in listFiles:
    img = readImage('data/health/'+elem)
    listResults += [0]
    listCancer += [img]

['1 (1).jpeg', '1 (1).jpg', '1 (1).png', '1 (10).jpeg', '1 (10).jpg']


In [12]:
print(listResults)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [13]:
import numpy as np
lst = np.random.permutation(len(listResults))
lst = list(lst)

In [14]:
lst

[32,
 43,
 155,
 84,
 123,
 198,
 91,
 57,
 98,
 54,
 108,
 197,
 110,
 153,
 157,
 171,
 163,
 15,
 190,
 89,
 162,
 62,
 95,
 115,
 127,
 63,
 152,
 4,
 53,
 151,
 174,
 142,
 85,
 167,
 158,
 136,
 188,
 5,
 27,
 67,
 113,
 16,
 193,
 83,
 126,
 173,
 176,
 81,
 44,
 88,
 184,
 185,
 93,
 186,
 187,
 1,
 147,
 168,
 33,
 7,
 10,
 45,
 72,
 129,
 114,
 160,
 73,
 166,
 58,
 179,
 189,
 140,
 76,
 172,
 9,
 2,
 195,
 65,
 165,
 20,
 182,
 119,
 39,
 19,
 46,
 128,
 164,
 11,
 40,
 47,
 60,
 146,
 130,
 52,
 86,
 180,
 194,
 181,
 51,
 90,
 34,
 31,
 77,
 26,
 21,
 131,
 30,
 112,
 183,
 50,
 145,
 13,
 196,
 64,
 55,
 107,
 133,
 36,
 118,
 24,
 94,
 78,
 29,
 37,
 161,
 80,
 66,
 69,
 12,
 17,
 82,
 143,
 41,
 169,
 106,
 120,
 109,
 99,
 125,
 121,
 6,
 141,
 148,
 177,
 8,
 87,
 124,
 150,
 14,
 132,
 35,
 103,
 191,
 18,
 154,
 56,
 22,
 25,
 97,
 144,
 122,
 79,
 23,
 101,
 96,
 156,
 139,
 42,
 138,
 74,
 0,
 104,
 71,
 92,
 49,
 75,
 170,
 175,
 199,
 38,
 149,
 68,
 59,
 192,


In [15]:
data = [listResults[i] for i in lst]

In [16]:
sz = (len(lst) * 80) // 100
train = [data[i] for i in range(sz)]
test = [data[i] for i in range(sz, len(lst))]
print(train)
print(test)

[1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0]
[0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0]


In [17]:
from statistics import mode
rsp = mode(train)
pred = []
for i in range(len(test)):
    pred+=[rsp]
acc = 0
for i in range(len(test)):
    if pred[i] == test[i]:
        acc+=1
acc = acc / len(test)
acc
precision = precision_score(test, pred, average='weighted')
recall = recall_score(test, pred, average='weighted')
print("acc: ", acc)
print("precision: ", precision)
print("recall: ", recall)

acc:  0.45
precision:  0.20249999999999999
recall:  0.45


c:\users\patcas rares\appdata\local\programs\python\python39\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
